In [50]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

import pickle 

## Read in Training Data

In [51]:
data = pd.read_csv('../Data/RYANDATA_filt.csv')
# data = pd.read_csv('RYANDATA_consolidated.csv')
print(list(data))
data.columns = ['V0','topic_split', 'topic', 'authors','title','journal','year','vol_issue','doi','abstract']
print(data.columns)

['Unnamed: 0', 'Catagory', 'Authors', 'Titles', 'Journals', 'Years', 'Vol_Isue', 'DOI', 'Abstract']
Index(['V0', 'topic', 'authors', 'title', 'journal', 'year', 'vol_issue',
       'doi', 'abstract'],
      dtype='object')


In [52]:
papers = pd.DataFrame(data['title'])
topic = pd.DataFrame(data['topic'])
author = pd.DataFrame(data['authors'])

In [53]:
print(len(papers))
print(len(topic))
data[['title','topic']].head()

35267
35267


,title,topic
0,Proximal radius fracture morphology following ...,BONE
1,Bone marrow transplantation for treatment of t...,BONE
2,(-)-epigallocatechin-3-gallate (egcg) enhances...,BONE
3,Biomechanical evaluation of a new fixation typ...,BONE
4,Pulsed electromagnetic fields: promising treat...,BONE


In [54]:
topic['topic'].unique() #consolidated


array(['BONE', 'CARDIOVASCULAR/CARDIOPULMONARY', 'CELLULAR/SUBCELLULAR',
       'COMPARATIVE', 'DENTAL/ORAL/FACIAL', 'ERGONOMICS',
       'GAIT/LOCOMOTION', 'JOINT/CARTILAGE', 'METHODS', 'MODELING',
       'MUSCLE', 'NEURAL', 'ORTHOPAEDICS/SPINE', 'ORTHOPAEDICS/SURGERY',
       'PROSTHETICS/ORTHOTICS', 'REHABILITATION', 'SPORT/EXERCISE',
       'TENDON/LIGAMENT', 'TISSUE/BIOMATERIAL', 'TRAUMA/IMPACTTESTING',
       'VETERINARY/AGRICULTURAL', 'VISUAL/VESTIBULAR/EYE',
       'EVOLUTION/ANTHROPOLOGY', 'ROBOTICS'], dtype=object)

In [55]:
from sklearn.preprocessing import LabelEncoder

feat = ['topic']
for x in feat:
    le = LabelEncoder()
    le.fit(list(topic[x].values))
    topic[x] = le.transform(list(topic[x]))

In [56]:
topic['topic'].unique()
data['topic_n'] = topic['topic']
data.head()

,V0,topic,authors,title,journal,year,vol_issue,doi,abstract,topic_n
0,0,BONE,"['Lacheta, L', 'Siebenlist, S', 'Lauber, M', '...",Proximal radius fracture morphology following ...,Bmc Musculoskeletal Disorders,Bmc Musculoskeletal Disorders,20:,http://dx.doi.org/10.1186/s12891-019-2529-9,common location articular fractures radial hea...,0
1,1,BONE,"['Lee, LR', 'Peacock, L', 'Ginn, SL', 'Cantril...",Bone marrow transplantation for treatment of t...,Calcified Tissue International,Calcified Tissue International,104:426-436,http://dx.doi.org/10.1007/s00223-018-0504-3,Bone marrow transplantation BMT healthy donor ...,0
2,2,BONE,"['Lin, SY', 'Kang, L', 'Chen, JC', 'Wang, CZ',...",(-)-epigallocatechin-3-gallate (egcg) enhances...,Phytomedicine,Phytomedicine,55:165-171,http://dx.doi.org/10.1016/j.phymed.2018.07.012,Previously found epigallocatechin 3 gallate EG...,0
3,3,BONE,"['Park, DW', 'Lim, A', 'Park, JW', 'Lim, KM', ...",Biomechanical evaluation of a new fixation typ...,Applied Sciences-Basel,Applied Sciences-Basel,9:,http://dx.doi.org/10.3390/app9050820,NaN,0
4,4,BONE,"['Wang, T', 'Yang, L', 'Jiang, J', 'Liu, Y', '...",Pulsed electromagnetic fields: promising treat...,Osteoporosis International,Osteoporosis International,30:267-276,http://dx.doi.org/10.1007/s00198-018-04822-6,Osteoporosis OP considered well defined diseas...,0


In [57]:
#save label encoder
np.save('../Models/Many_ML_models/test_many_ML_Label_Encoder.npy',le.classes_)
print('Saved label encoder: test_many_ML_Label_Encoder.npy')

Saved label encoder: test_many_ML_Label_Encoder.npy


In [58]:
data['everything'] = pd.DataFrame(data['title'])
data['everything'] = data['everything'].replace(np.nan, '', regex = True)
print(data['everything'][0])

Proximal radius fracture morphology following axial force impact: a biomechanical evaluation of fracture patterns


## Clean up titles

In [59]:
def change(t):
    t = t.split()
    return ' '.join([(i) for (i) in t if i not in stop])
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
stop.append('The')
stop.append('An')
stop.append('A')
stop.append('Do')
stop.append('Is')
stop.append('In')

data['everything'].apply(change).head()


0    Proximal radius fracture morphology following ...
1    Bone marrow transplantation treatment col1a2(+...
2    (-)-epigallocatechin-3-gallate (egcg) enhances...
3    Biomechanical evaluation new fixation type 3d-...
4    Pulsed electromagnetic fields: promising treat...
Name: everything, dtype: object

## Split Data

In [60]:
#split then vectorize per https://www.ritchieng.com/machine-learning-multinomial-naive-bayes-vectorization/#3.-Reading-a-text-based-dataset-into-pandas 
#(scroll down to 'Why are we splitting into training and testing sets before vectorizing?')

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced
# from sklearn.model_selection import train_test_split


# X_train, X_test, y_train, y_test = train_test_split(data['everything'],
#                                                     data['topic'],
#                                                     test_size=0.2,
#                                                     random_state = 0)

from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits= 1, 
                             test_size = 0.2, 
                             random_state = 0)

for train_idx, test_idx in sss.split(data['everything'],data['topic']):
    X_train, X_test = data['everything'][train_idx], data['everything'][test_idx]
    y_train, y_test = data['topic'][train_idx], data['topic'][test_idx]



# print (X_train.shape)
# print (X_test.shape)
# print (y_train.shape)
# print (y_test.shape)


## Vectorize words in titles

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=3, #min occurances needed
                             max_df=.75, #max occuraces allowed (%)
                             ngram_range=(1,2), #size range of grams (1-2 words)
                             strip_accents='unicode',
                             lowercase =True,
                             analyzer='word', 
                             token_pattern=r'\w+', 
                             stop_words = 'english',
                             smooth_idf = True
                             )
X_train_dtm = vectorizer.fit_transform(X_train)
X_test_dtm = vectorizer.transform(X_test) #DONT FIT VECTORIZER TO TEST DATA. ONLY TRANSFORM
X_all_dtm = vectorizer.transform(data['everything'])

with open('../Models/Many_ML_models/Many_ml_Vectorizer_tdif.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
    
print(X_train_dtm.shape)
print(X_test_dtm.shape)
print(X_all_dtm.shape)

(28213, 22527)
(7054, 22527)
(35267, 22527)


In [62]:
# RESAMPLE TO BALANCE CLASSES
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train_dtm, y_train)
from collections import Counter
# print(sorted(Counter(y_resampled).items()))
print('Each Topic has ' + str(Counter(y_resampled)['BONE']) +' entries.')

Each Topic has 2547 entries.


# Test Models: 
## Multinomial Naive Bayes

In [63]:
mnb = MultinomialNB(alpha=.45)
mnb.fit(X_resampled, y_resampled)
pred = mnb.predict(X_test_dtm)
print('MNB f1 score: ' + str(np.round(metrics.f1_score(y_test, pred, average='macro')*100,1)))
print('MNB accuracy: ' + str(np.round(metrics.accuracy_score(y_test, pred)*100,1)))
# print(classification_report_imbalanced(y_test, pred))

with open('../Models/Many_ML_models/mnb.pkl', 'wb') as file:  
    pickle.dump(mnb, file)
print('Saved Multinomial Naive Bayes: mnb.pkl')

MNB f1 score: 63.5
MNB accuracy: 64.5
Saved Multinomial Naive Bayes: mnb.pkl


## Logistic Regression

In [64]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(solver= 'sag',max_iter=500,random_state=0, multi_class='multinomial')
logreg.fit(X_resampled, y_resampled)
pred = logreg.predict(X_test_dtm)
print('LogReg f1 score: ' + str(np.round(metrics.f1_score(y_test, pred, average='macro')*100,1)))
print('LogReg f1 score: ' + str(np.round(metrics.accuracy_score(y_test, pred)*100,1)))
# print(classification_report_imbalanced(y_test, pred))
                                
with open('../Models/Many_ML_models/logreg.pkl', 'wb') as file:  
    pickle.dump(logreg, file)
print('Saved Logistic Regression: logreg.pkl')

LogReg f1 score: 66.9
LogReg f1 score: 67.4
Saved Logistic Regression: logreg.pkl


## Stochastic Gradient Descent (SGD) learning

In [66]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(max_iter=1000, 
                    random_state=0, 
                    tol = 1e-3, 
                    alpha= 0.0001
                   )
sgd.fit(X_resampled, y_resampled)
pred = sgd.predict(X_test_dtm)
print('SGD f1 score: ' + str(np.round(metrics.f1_score(y_test, pred, average='macro')*100,1)))
print('SGD f1 score: ' + str(np.round(metrics.accuracy_score(y_test, pred)*100,1)))
# print(classification_report_imbalanced(y_test, pred))
                             
with open('../Models/Many_ML_models/sgd.pkl', 'wb') as file:  
    pickle.dump(sgd, file)
print('Saved Stochastic Gradient Descent (SGD): sgd.pkl')

SGD f1 score: 63.8
SGD f1 score: 65.0
Saved Stochastic Gradient Descent (SGD): sgd.pkl


## Linear SVC

In [67]:
from sklearn.svm import LinearSVC

#fit model
lsvc =LinearSVC(max_iter=1000)
lsvc.fit(X_resampled, y_resampled)
pred = lsvc.predict(X_test_dtm)
print('LSVC f1 score: ' + str(np.round(metrics.f1_score(y_test, pred, average='macro')*100,1)))
print('LSVC f1 score: ' + str(np.round(metrics.accuracy_score(y_test, pred)*100,1)))
# print(classification_report_imbalanced(y_test, pred))
                              
with open('../Models/Many_ML_models/lsvc.pkl', 'wb') as file:  
    pickle.dump(lsvc, file)
print('Saved Linear Supper Vector Classifier (SVC): svc.pkl')

LSVC f1 score: 63.5
LSVC f1 score: 64.6
Saved Linear Supper Vector Classifier (SVC): svc.pkl


# Neural Network  kfold Cross Validation w/ balanced dataset

In [68]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
#https://stackoverflow.com/questions/44683197/implement-k-fold-cross-validation-in-mlpclassification-python
# VECTORIZE
X_all_v = vectorizer.fit_transform(data['everything'])


In [69]:
# RESAMPLE TO BALANCE CLASSES
X_all_resampled, y_all_resampled = ros.fit_resample(X_train_dtm, y_train)
from collections import Counter
#print(sorted(Counter(y_resampled).items()))
#print(sorted(Counter(y_resampled['BONE']).items()))
print('Number of papers per topic: ' + str(Counter(y_resampled)['BONE']))

Number of papers per topic: 2547


In [72]:
#BUILD MODEL/FOLDS
kf = KFold(n_splits = 5)
mlp = MLPClassifier(hidden_layer_sizes = (200),
                    activation = 'relu',
                    solver = 'adam',
                    batch_size = 1000,
                    learning_rate = 'adaptive',
                    shuffle = True,
                    random_state = 0,
                    verbose = 0,
                    early_stopping = True,
                    validation_fraction= 0.2,
                    max_iter= 25,
                    n_iter_no_change=5,
                    tol = 0.01)
import time
for train_indices, test_indices in kf.split(X_resampled):
    mlp.fit(X_resampled[train_indices], y_resampled[train_indices])
    print('Accuracy: ' + str(np.round(mlp.score(X_resampled[test_indices], y_resampled[test_indices])*100,1)))
    time.sleep(5)
          
with open('../Models/Many_ML_models/mlp.pkl', 'wb') as file:  
    pickle.dump(mlp, file)
print('Saved MLP Classifier: mlp.pkl')

Accuracy: 80.8
Accuracy: 80.5
Accuracy: 86.9
Accuracy: 88.4
Accuracy: 91.1
Saved MLP Classifier: mlp.pkl


In [73]:
#K(5) fold cross validation scores: 
scores_print = [0.8035887850467289,
0.7960373831775701,
0.8559892328398385,
0.8618962165395544,
0.9078809630626589]

print('MEAN:',np.mean(scores_print))
print('SD:',np.std(scores_print))

MEAN: 0.8450785161332701
SD: 0.04116419957569286


In [74]:
print('X_test shape: ' + str(X_test_dtm.shape))
print('y_test shape: ' + str(y_test.shape))

X_test shape: (7054, 22527)
y_test shape: (7054,)


In [66]:
#print(X_test_dtm)
pred = mlp.predict(X_test_dtm)
print(classification_report_imbalanced(y_test, pred))

                                      pre       rec       spe        f1       geo       iba       sup

                          BONE       0.70      0.74      0.98      0.72      0.85      0.71       392
CARDIOVASCULAR/CARDIOPULMONARY       0.72      0.67      0.99      0.69      0.81      0.64       233
          CELLULAR/SUBCELLULAR       0.64      0.72      0.99      0.68      0.84      0.69       243
                   COMPARATIVE       0.52      0.54      0.98      0.53      0.72      0.50       321
            DENTAL/ORAL/FACIAL       0.78      0.73      0.99      0.75      0.85      0.71       269
                    ERGONOMICS       0.62      0.58      0.99      0.60      0.76      0.55       100
        EVOLUTION/ANTHROPOLOGY       0.62      0.59      0.99      0.61      0.77      0.57       200
               GAIT/LOCOMOTION       0.61      0.65      0.96      0.63      0.79      0.60       637
               JOINT/CARTILAGE       0.56      0.51      0.98      0.54      0.71

# Plot Confusion Matrix 

In [75]:
import matplotlib.pyplot as plt
import seaborn as sns

data['topic_id'] = data['topic'].factorize()[0]
topic_id_df = data[['topic',
                  'topic_id']].drop_duplicates().sort_values('topic_id')
topic_to_id = dict(topic_id_df.values)
id_to_topic = dict(topic_id_df[['topic_id','topic']].values)

model = mlp

pred = mlp.predict(X_test_dtm)
y_pred = pred
from sklearn.metrics import confusion_matrix
plt.rc('xtick',labelsize = 20)
plt.rc('ytick',labelsize = 20)
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat_rowsum = [sum(row) for row in conf_mat]
conf_mat_perc = conf_mat / conf_mat_rowsum
fig, ax = plt.subplots(figsize=(16,9))
sns.set(font_scale=1.2) #font size multiplier
sns.heatmap(conf_mat_perc, annot=True, fmt='.0%', cmap = 'magma', annot_kws={"size": 12},
            xticklabels=topic_id_df.topic.values, yticklabels=topic_id_df.topic.values)

plt.ylabel('Actual',fontsize = 20)
plt.xlabel('Predicted',fontsize = 20)
plt.yticks(size = 12)
plt.xticks(size = 12, rotation=30,ha='right')
plt.title('Percent Predicted Correct', fontsize = 26)
plt.yticks( rotation='horizontal')
fig.tight_layout(pad = 2)
plt.show()
print('Accuracy: ' + str(round(sum(np.diagonal(conf_mat))/X_test.shape[0]*100,1)) + '%')
fig.savefig('../Plots/biomchL_predict_plot_MLP.png')

<Figure size 1600x900 with 2 Axes>

Accuracy: 64.1%


# Table of Classification Scores

# Test model on string

In [419]:
text = ['Stroller running: Energetic and kinematic changes across pushing methods']
text[0] = text[0].lower()
#text = list(text)
s = (vectorizer.transform(text))
#s = vectorizer.fit_transform(df)
print (s.shape)
d = (logreg.predict(s))
print(d)
# le.inverse_transform(d)[0]

(1, 23613)
['SPORT/EXERCISE']


# Save a model for later

In [420]:
# from sklearn.externals import joblib
# joblib.dump(mlp, '../Models/NeuralNet.pkl')
# print ("Model Saved")

Model Saved


# Save a vectorizer for later

In [421]:
# from sklearn.externals import joblib
# joblib.dump(vectorizer, '../Models/test_many_ML_vectorizer.pkl')
# print('Vectorizer Saved')

Vectorizer Saved


# Load a model

In [ ]:
# clf = joblib.load('../Models/MultNB.pkl')
# clf